# **Using NMF on sparse data**


## Step 1:
I will load the movie ratings sample data from the MovieLens dataset and use sklearn matrix factorization to predict the missing ratings from the test data and measure the RMSE.

To account for missing data, I am imputing all of a user's unrated movies to a 3. For the number of latent factors, I will use the number of genre columns in the movies table (18).

In [44]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import csr_matrix

In [45]:
MV_users = pd.read_csv('movie-data/users.csv')
MV_movies = pd.read_csv('movie-data/movies.csv')
train = pd.read_csv('movie-data/train.csv')
test = pd.read_csv('movie-data/test.csv')

In [46]:
num_users = MV_users['uID'].max()
num_movies = MV_movies['mID'].max()

# Create a full user-movie matrix of threes
user_movie_matrix_train = np.full((num_users, num_movies), 3.0)

# Populate the matrix with ratings from the training data
for _, row in train.iterrows():
    # Adjust for 0-based indexing if IDs start from 1
    user_idx = row['uID'] - 1
    movie_idx = row['mID'] - 1
    rating = row['rating']
    if user_idx < num_users and movie_idx < num_movies:
        user_movie_matrix_train[user_idx, movie_idx] = rating

# Create a sparse matrix for NMF for efficiency
user_movie_sparse_train = csr_matrix(user_movie_matrix_train)

# For the number of latent factors, I am using the number of genre columns in the movies table.
n_components = 18

# Initialize and fit the NMF model on the training data matrix
nmf_model = NMF(n_components=n_components, init='nndsvda', max_iter=1000, random_state=42)
user_factors = nmf_model.fit_transform(user_movie_sparse_train)
movie_factors = nmf_model.components_

# Reconstruct the matrix to predict missing ratings for the entire matrix
predicted_ratings_full_matrix = np.dot(user_factors, movie_factors)

# Predict ratings for the test data
test_predictions = []
for _, row in test.iterrows():
    # Adjust for 0-based indexing
    user_idx = row['uID'] - 1
    movie_idx = row['mID'] - 1
    predicted_rating = predicted_ratings_full_matrix[user_idx, movie_idx]
    test_predictions.append(predicted_rating)

# Calculate RMSE on the test data
test_true_ratings = test['rating'].values
rmse = sqrt(mean_squared_error(test_true_ratings, test_predictions))

print(f"NMF RMSE with imputed threes on test set: {rmse:.4f}")

NMF RMSE with imputed threes on test set: 1.1062


This model took an excruciatingly long time to run and had a RMSE worse than all my collaborative models for this data set.

## Step 2:
Now I will discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to the simple baseline or similarity-based methods I did in an earlier model, and suggest a way(s) to fix it.

### Why sklearn.decomposition.NMF performs poorly with this data
Since sklearn's NMF requires a complete matrix, I had to fill missing values with something. For simplicity, I chose 3s; it's possible that imputing a user's average movie rating to unrated movies would have worked better. But whatever number we imputed for unrated movie, we would still be forced by the model type to "learn" from fake data. Collaborative methods don't have this problem.

Our NMF model also lacks user and movie bias terms, such as a user's tendency to rate movies higher or lower, and a movie's tendency to receive higher or lower ratings. Collaborative filtering can capture these biases.

Additionally, my choice of latent features, while informed by the data, is arbitrary and led to a very, very long run time. Given that the performance was only slightly better than the baseline methods, this approach might not be worth it.


### How to improve the model
In terms of improving the sklearn NMF performance itself, I would need to find ways to improve the imputed ratings and bias, and speed up performance. 

It's possible that performance could be sped up by minimizing the number of latent features to what's aboslutely necessary. But I would have to run and test the model many, many times to get the best number.

The imputed ratings could be changed to the user average, the movie average, or some mathematical formula based on those. This would inherently correct for some bias.

There might also be a gradient descent method I could use to correct for user and movie bias. This would involve iterating many times of the data set.

I could also try a singular valye deconposition, which could operate directly on the sparse matrix without rating imputation.